In [11]:
import numpy as np
import time

# Timing for the different functions of the code

## Hermite polynomials

Comparison of the recursive and scipy Hermite polynomials

In [2]:
from scipy import special

def hermite_pol_recursive(x, n):
    if n == 0:
        return 1
    elif n == 1:
        return 2*x
    else:
        return 2*x*hermite_pol_recursive(x, n-1) - 2*(n - 1)*hermite_pol_recursive(x, n-2)

def hermite_pol_scipy_1(x, n):
    H = special.hermite(n)
    return H(x)

def hermite_pol_scipy_2(x, n):
    return special.eval_hermite(n, x)

In [3]:
N = 10000
x = np.linspace(-10, 10, N)

print("Recursive Hermite polynomials")
%timeit hermite_pol_recursive(x, 1)
%timeit hermite_pol_recursive(x, 5)
%timeit hermite_pol_recursive(x, 10)

print("Scipy Hermite polynomials (option 1)")
%timeit hermite_pol_scipy_1(x, 1)
%timeit hermite_pol_scipy_1(x, 5)
%timeit hermite_pol_scipy_1(x, 10)

print("Scipy Hermite polynomials (option 2)")
%timeit hermite_pol_scipy_2(x, 1)
%timeit hermite_pol_scipy_2(x, 5)
%timeit hermite_pol_scipy_2(x, 10)

Recursive Hermite polynomials
3.59 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
129 µs ± 996 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.64 ms ± 7.57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Scipy Hermite polynomials (option 1)
291 µs ± 6.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
350 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
445 µs ± 3.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Scipy Hermite polynomials (option 2)
186 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
215 µs ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
277 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Timing for the different functions of the code

Write necessary functions

In [4]:
import lib
import numpy as np
from scipy import special
import os
import matplotlib.pyplot as plt

In [5]:
lz = 0.5
lx = 0.75

OMEGA_X = 1/lx**2
OMEGA_Z = 1/lz**2

def Hermite_pol(x, n):
    """
    Evaluates the Hermite polynomial of order n at x.

    Parameters
    ==========
    x : float or np.ndarray
        Values in which to evaluate the Hermite polynomial
    n : int
        Oder of the Hermite polynomial

    Returns
    =======
    float or np.ndarray
    """
    return special.eval_hermite(n, x)

def HO_wf(x, n, omega=OMEGA_X):
    """
    Evaluates the n^th eigenfunction of the Harmonic Oscillator at x.
    The units of x are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x : float or np.ndarray(N)
        Values in which to evaluate eigenfunction
    n : int
        Number of the eigenfunction
    omega : float
        Angular frequency of the Harmonic Oscillator

    Returns
    =======
    float or np.ndarray(N)
    """
    #return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) * np.exp(-omega*x**2 / 2) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))
    return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))

def HO_wf_3D(x, y, z, nx, ny, nz, omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z):
    """
    Evaluates the n^th eigenfunction of the (anisotropic) Harmonic Oscillator at x,y,z.
    The units of x,y,z are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x, y, z : float or np.ndarray(N)
        Position in which to evaluate the wave function
    nx, ny, nz : int
        Number of the eigenfunction for each cartesian coordinate
    omega_x, omega_y, omega_z : float
        Harmonic osciallation constant for each cartesian coordinate

    Returns
    =======
    float or np.ndarray(N)
    """
    return HO_wf(x, nx, omega=omega_x)*HO_wf(y, ny, omega=omega_y)*HO_wf(z, nz, omega=omega_z)

def index_to_q_numbers(k):
    """
    Returns the quantum numbers nx, ny, nz associated with the basis index k

    Parameters
    ----------
    k: int
        Index of the basis from 0 to 13

    Returns
    ----------
    nz ,ny, nz : int 
        Quantum numbers
    """

    q_numbers = np.array([(0,0,0),(0,1,0),(1,0,0),(1,1,0),(2,0,0),(0,2,0),(2,1,0),(1,2,0),(3,0,0),(0,3,0),(2,2,0),(2,2,0),(3,1,0),(1,3,0)])
    #q_numbers = np.array([(0,0,0),(0,0,1),(0,1,0),(1,0,0),(0,1,1),(1,0,1),(1,1,0),(1,1,1),(1,1,2),(1,2,1),(2,1,1),(1,2,2),(2,1,2),(2,2,1)])

    return q_numbers[k]

In [6]:
def integrand_2(R, p, r, q, s):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    
    x1 = R[:,0]
    y1 = R[:,1]
    z1 = R[:,2]
    x2 = R[:,3]
    y2 = R[:,4]
    z2 = R[:,5]
    
    n_p = index_to_q_numbers(p-1)
    n_r = index_to_q_numbers(r-1)
    n_q = index_to_q_numbers(q-1)
    n_s = index_to_q_numbers(s-1)
    
    result = 1/r12
    
    result *= HO_wf_3D(x1, y1, z1, n_p[0], n_p[1], n_p[2])
    result *= HO_wf_3D(x2, y2, z2, n_r[0], n_r[1], n_r[2])
    result *= HO_wf_3D(x1, y1, z1, n_q[0], n_q[1], n_q[2])
    result *= HO_wf_3D(x2, y2, z2, n_s[0], n_s[1], n_s[2])
  
    
    return result

def norm_product(p, r, q, s):
    return (np.pi/OMEGA_X)**2 * np.pi/OMEGA_Z

def two_body_integrand(p, r, q, s):
    f = lambda R: integrand_2(R, p, r, q, s)*norm_product(p, r, q, s)
    
    return f

In [7]:
def analytical_1(p,q):
    n_p = index_to_q_numbers(p-1)
    I = (p==q)*(OMEGA_X*(n_p[0] + n_p[1] + 1) + OMEGA_Z*(n_p[2] + 0.5))
    
    return I

def f_cov(p, r, q, s):
    
    cov = 0.5*np.diag([1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z, 1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z])
    
    return cov

In [8]:
def time_integrals(N_basis, N_points):
    integrals_file = "integrals_QD_timing.npy"

    normalized_wf = True
    
    # One- and Two-body integrals
    MC_args = {"f_cov":f_cov, "f_integrand":two_body_integrand, "N_points":N_points}
    integrals = lib.integral_master(N_basis)
    integrals.calculate(integrals_file, analytical_1, MC_args = MC_args)

### Timing integrals

In [14]:
# Time integrals
N_points = 100000
N_basis = 14

print("Complete integral dictionary for N_points =", N_points, "and N_basis =", N_basis)
t0 = time.time()
time_integrals(N_basis, N_points)
t1 = time.time()
print("Total time:", t1-t0)

Complete integral dictionary for N_points = 100000 and N_basis = 14
Calculating two electron integrals: 1/15
Calculating two electron integrals: 2/15
Calculating two electron integrals: 3/15
Calculating two electron integrals: 4/15
Calculating two electron integrals: 5/15
Calculating two electron integrals: 6/15
Calculating two electron integrals: 7/15
Calculating two electron integrals: 8/15
Calculating two electron integrals: 9/15
Calculating two electron integrals: 10/15
Calculating two electron integrals: 11/15
Calculating two electron integrals: 12/15
Calculating two electron integrals: 13/15
Calculating two electron integrals: 14/15
Total time: 557.6338496208191


Total time for N_points = 100000 and N_basis = 14 is T = 557.6338496208191s = 9 min 18 s

### Timing Self-Consistent Field

In [15]:
def time_SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF):
    lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)

In [18]:
# Time SCF
N_basis = 14

MC_args = {"f_cov":f_cov, "f_integrand":two_body_integrand, "N_points":N_points}
integrals_file = "integrals_QD_timing.npy"
integrals = lib.integral_master(N_basis)
integrals.calculate(integrals_file, analytical_1, MC_args = MC_args)
S = np.eye(N_basis)
max_iter_SCF = 500
eps_SCF = 1e-4

N_electrons = 2
print("Self-Consistent Field for  N_electrons =", N_electrons)
t0 = time.time()
time_SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)
t1 = time.time()
print("Total time:", t1-t0)

N_electrons = 14
print("Self-Consistent Field for  N_electrons =", N_electrons)
t0 = time.time()
time_SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)
t1 = time.time()
print("Total time:", t1-t0)

Integral file already exists. Not computing the integrals. 
Self-Consistent Field for  N_electrons = 2
E =  4.478288693412502  | N(SCF) = 1
E =  5.579846587725906  | N(SCF) = 2
E =  5.865550227088004  | N(SCF) = 3
E =  6.172709450453027  | N(SCF) = 4
E =  6.245892552574776  | N(SCF) = 5
E =  6.481378292703055  | N(SCF) = 6
E =  6.255342165420831  | N(SCF) = 7
E =  6.3159844416422075  | N(SCF) = 8
E =  6.145205216028046  | N(SCF) = 9
E =  6.363625974454621  | N(SCF) = 10
E =  6.194354186199688  | N(SCF) = 11
E =  6.536571854917926  | N(SCF) = 12
E =  6.244311790202869  | N(SCF) = 13
E =  6.229181545304552  | N(SCF) = 14
E =  6.355262453667871  | N(SCF) = 15
E =  6.247257972821325  | N(SCF) = 16
E =  6.270732458129164  | N(SCF) = 17
E =  6.297813008119926  | N(SCF) = 18
E =  6.414412734715823  | N(SCF) = 19
E =  6.272977446868636  | N(SCF) = 20
E =  6.323682185804683  | N(SCF) = 21
E =  6.299492525987057  | N(SCF) = 22
E =  6.24707334542691  | N(SCF) = 23
E =  6.417664529036414  | N(SCF)

Total time for N_electrons = 2 is T = 5 s and for N_electrons = 14 is T = 4 s

In [ ]:
def time_SCF_average(N_electrons, integrals, S, max_iter_SCF, eps_SCF):
    lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)